In [2]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

In [3]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [4]:
# Read in the data
train = list(read_dataset("../data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [8]:
# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [9]:
# Define the model
W_sm = model.add_lookup_parameters((nwords, ntags)) # Word weights
b_sm = model.add_parameters((ntags))                # Softmax bias

In [12]:
# A function to calculate scores for one value
def calc_scores(words):
  dy.renew_cg()
  score = dy.esum([dy.lookup(W_sm, x) for x in words])
  #b_sm_exp = dy.parameter(b_sm)
  return score + b_sm

In [15]:
for ITER in range(10):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=0.1319, time=0.30s
iter 0: test acc=0.3805
iter 1: train loss/sent=0.1314, time=0.26s
iter 1: test acc=0.3814
iter 2: train loss/sent=0.1310, time=0.26s
iter 2: test acc=0.3824
iter 3: train loss/sent=0.1306, time=0.28s
iter 3: test acc=0.3814
iter 4: train loss/sent=0.1303, time=0.26s
iter 4: test acc=0.3860
iter 5: train loss/sent=0.1299, time=0.26s
iter 5: test acc=0.3837
iter 6: train loss/sent=0.1294, time=0.25s
iter 6: test acc=0.3810
iter 7: train loss/sent=0.1292, time=0.25s
iter 7: test acc=0.3778
iter 8: train loss/sent=0.1288, time=0.25s
iter 8: test acc=0.3805
iter 9: train loss/sent=0.1283, time=0.25s
iter 9: test acc=0.3814
